In [1]:
!pip install fastapi uvicorn pyngrok requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00


In [2]:
import requests
from fastapi import FastAPI
from pyngrok import ngrok

In [3]:
def evaluate_url(url):
    """
    Function to evaluate the credibility of a given URL.
    It assigns a score based on heuristics (domain authority, SSL, etc.).
    """
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            score = 0.9  # Mock credibility score for working URLs
            explanation = "This source is considered credible based on accessibility."
        else:
            score = 0.3
            explanation = "The source is inaccessible or unreliable."
    except requests.exceptions.RequestException:
        score = 0.0
        explanation = "The source could not be reached."

    return {"score": score, "explanation": explanation}

In [4]:
app = FastAPI()

@app.get("/check_source")
def check_source(url: str):
    """
    API endpoint to evaluate a URL.
    """
    result = evaluate_url(url)
    return {"url": url, "credibility_score": result["score"], "explanation": result["explanation"]}

# Start FastAPI server in the background
import uvicorn
from threading import Thread

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = Thread(target=run)
thread.start()

In [6]:
!ngrok config add-authtoken 2v6cooAxovgiOFCJsb3a5ePHrFv_6KPhKyxW19oE4wf3Xafmo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
public_url = ngrok.connect(8000).public_url
print(f"Public API URL: {public_url}/check_source?url=https://www.nytimes.com")


Public API URL: https://dabf-34-41-243-117.ngrok-free.app/check_source?url=https://www.nytimes.com


In [8]:
api_url = public_url + "/check_source?url=https://www.nytimes.com"
response = requests.get(api_url).json()
print(response)

INFO:     34.41.243.117:0 - "GET /check_source?url=https://www.nytimes.com HTTP/1.1" 200 OK
{'url': 'https://www.nytimes.com', 'credibility_score': 0.9, 'explanation': 'This source is considered credible based on accessibility.'}
